In [1]:
import pandas as pd
import numpy as np
import re
import pickle
from datetime import datetime

In [2]:
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
import plotly.io as pio
import plotly.plotly as py
import plotly.graph_objs as go
from plotly.graph_objs import *

In [3]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score,roc_curve, roc_auc_score, auc
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, BaggingClassifier
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB, BernoulliNB, GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, cross_val_score

In [4]:
# Define the color palette (17 colors).
Viridis= ['#440154', '#48186a', '#472d7b', '#424086', '#3b528b', '#33638d', '#2c728e', '#26828e', '#21918c', '#1fa088',
          '#28ae80', '#3fbc73', '#5ec962', '#84d44b', '#addc30','#d8e219', '#fde725']   

In [5]:
# read clean datafile
df = pd.read_csv('../data/dataset5.gz', compression='gzip', header=0, sep=',', quotechar='"')
print(df.shape)

(93701, 39)


With this many predictors, overfitting is definitely a risk. We'll take 5 steps to avoid overfit:
* standardize the predictors (i.e., scale variables to between 0 and 1)
* regularization (apply a penalty to large coefficients)
* apply kfold cross-validation
* use gridsearch to determine the optimal hyperparameters for our model
* compare multiple models and select the one with the best evaluation metrics

## Dealing with imbalanced classes in the training dataset

Because our target variable (failure) is highly imbalanced, we need to take several precautions to accurately evaluate our model. Because the data is overwhelmingly positive, a model that predicted "no failure" would be accurate 99% of the time! We will take two steps to deal with imbalance:  
* Up-sample Minority Class
* Use ROC-AUC and F1 as our performance metric rather than accuracy  
https://elitedatascience.com/imbalanced-classes

Because we need to resample, we'll conduct train-test split _before_ separating our X and y data. This is because we only want to oversample the training data, not the testing data! In addition, if we conducted oversampling _before_ the train-test split, we would almost certainly have duplicate data in both the training and testing sets, which would result in severe overfit.

In [47]:
# Split the data
df_train, df_test = train_test_split(df, test_size = .3, random_state=42)

There are multiple methods for oversampling the training data. One technique available in scikit learn is [SMOTE - Synthetic Minority Over-sampling Technique](https://imbalanced-learn.readthedocs.io/en/stable/generated/imblearn.over_sampling.SMOTE.html), but we'll just use random sampling with replacement (i.e., resampling).

In [8]:
from sklearn.utils import resample
# Separate majority and minority classes
df_majority = df_train[df_train['failure']==0]
df_minority = df_train[df_train['failure']==1]
 
# Upsample minority class
df_minority_upsampled = resample(df_minority, 
                                 replace=True,     # sample with replacement
                                 n_samples=len(df_majority),    # to match majority class
                                 random_state=42) # reproducible results
 
# Combine majority class with upsampled minority class
df_train_upsampled = pd.concat([df_majority, df_minority_upsampled])
 
# Display new class counts
df_train_upsampled['failure'].value_counts()
# hat tip: https://elitedatascience.com/imbalanced-classes

1    65524
0    65524
Name: failure, dtype: int64

We'll use `y1_test` later on, for evaluation purposes.

In [9]:
# final x-values (features)
X_train=df_train_upsampled.drop(['failure', 'device'], axis=1)
X_test=df_test.drop(['failure', 'device'], axis=1)

In [10]:
# final y-values
y_train=df_train_upsampled['failure'].values
y1_test=df_test[['device','failure']] # Hold onto the device variable for later use, but remove it from the modeling data.
y_test=df_test['failure'].values

In [11]:
# confirm that lengths match
assert len(X_train)==len(y_train)
assert len(X_test)==len(y_test)

# Preliminary model exploration

As a preliminary step, we'll take a look at four popular classifiers (naive bayes, KNN, logistic, and random forest) and see how they handle our data using the default hyperparameters. This will help us determine which model to select for further tuning.

### Naive Bayes Classifier

In [12]:
gnb = GaussianNB()
# Fit on the training data
gnb_model = gnb.fit(X_train, y_train)
# Predict on the testing data
predictions=gnb_model.predict(X_test)
probabilities = gnb_model.predict_proba(X_test)[:,1]
# Calculate the roc-auc score
auc_nb=metrics.roc_auc_score(y_test, predictions)
acc_nb = metrics.accuracy_score(y_test, predictions)
f1_nb = metrics.f1_score(y_test, predictions)
# Display
print('F1 Score', "%.4f" % round(f1_nb,4))
print('Accuracy', "%.4f" % round(acc_nb,4))
print('AUC Score', "%.4f" % round(auc_nb,4))

F1 Score 0.0045
Accuracy 0.7784
AUC Score 0.6693


### Logistic Regression

In [13]:
logreg = LogisticRegression()
# Fit on the training data
log_model=logreg.fit(X_train, y_train)
# Predict on the testing data
predictions=log_model.predict(X_test)
probabilities = log_model.predict_proba(X_test)[:,1]
# Calculate the roc-auc score
auc_log=metrics.roc_auc_score(y_test, predictions)
acc_log = metrics.accuracy_score(y_test, predictions)
f1_log = metrics.f1_score(y_test, predictions)
# Display
print('F1 Score', "%.4f" % round(f1_log,4))
print('Accuracy', "%.4f" % round(acc_log,4))
print('AUC Score', "%.4f" % round(auc_log,4))

F1 Score 0.0109
Accuracy 0.8901
AUC Score 0.7852


### KNN Classifier

In [14]:
knn = KNeighborsClassifier(n_neighbors=7)
# Fit on the training data
knn_model=knn.fit(X_train, y_train)
# Predict on the testing data
predictions=knn_model.predict(X_test)
probabilities = knn_model.predict_proba(X_test)[:,1]
# Calculate the roc-auc score
auc_knn=metrics.roc_auc_score(y_test, predictions)
acc_knn = metrics.accuracy_score(y_test, predictions)
f1_knn = metrics.f1_score(y_test, predictions)
# Display
print('F1 Score', "%.4f" % round(f1_knn,4))
print('Accuracy', "%.4f" % round(acc_knn,4))
print('AUC Score', "%.4f" % round(auc_knn,4))

F1 Score 0.0202
Accuracy 0.9965
AUC Score 0.5187


### Random Forest

In [15]:
rf = RandomForestClassifier()
# Fit on the training data
rf_model=rf.fit(X_train, y_train)
# Predict on the testing data
predictions=rf_model.predict(X_test)
probabilities = rf_model.predict_proba(X_test)[:,1]
# Calculate the roc-auc score
auc_rf=metrics.roc_auc_score(y_test, predictions)
acc_rf = metrics.accuracy_score(y_test, predictions)
f1_rf = metrics.f1_score(y_test, predictions)
# Display
print('F1 Score', "%.4f" % round(f1_rf,4))
print('Accuracy', "%.4f" % round(acc_rf,4))
print('AUC Score', "%.4f" % round(auc_rf,4))

F1 Score 0.0000
Accuracy 0.9990
AUC Score 0.4999


### Comparison of Four Models

At first glance, it appears that the `random forest` and `KNN` models are the best - they have accuracy scores of nearly 100%. But of course this is misleading - imbalanced classes typically result in high accuracy, since this score simply measures the number of correct predictions over the number of all test observations.

A better metric for comparison in this case is the ["receiver operator characteristic - area under the curve"](https://towardsdatascience.com/understanding-auc-roc-curve-68b2303cc9c5) or `ROC AUC` score. It measures the balance between the true positive rate and the false positive rate.
* true positive rate: TP/ (TP + FN) -- also known as `recall`
* false positive rate: FP / (TN + FP)

Another metric to keep in mind is the F1 score. This score ranges from 0 to 1 and is an overall measure of a model’s accuracy that combines recall and precision. A higher F1 score means that the model has low false positives and low false negatives. Our models have an F1 score near zero, which is why we'll take a look at neural nets next.
* precision: TP/ (TP + FP)

In [16]:
f1=[f1_nb, f1_log, f1_knn, f1_rf]
acc=[acc_nb, acc_log, acc_knn, acc_rf]
auc=[auc_nb, auc_log, auc_knn, auc_rf]
models=['naive bayes', 'logistic regression', 'k-nearest neighbors', 'random forest']
index=['F1 score', 'Accuracy', 'AUC score']
results=pd.DataFrame([f1, acc, auc], index=index, columns=models)

In [17]:
results.head()

,naive bayes,logistic regression,k-nearest neighbors,random forest
F1 score,0.004476,0.010887,0.020202,0.000000
Accuracy,0.778450,0.890114,0.996549,0.999004
AUC score,0.669322,0.785151,0.518700,0.499947


In [18]:
# Pickle those results for comparison with tensorflow.
with open('model_metrics.pkl', 'wb') as f:
    pickle.dump(results, f)
f.close()

In [19]:
# Let's display that with plotly.
mydata1 = go.Bar(
    x=results.loc['F1 score'].index,
    y=results.loc['F1 score'],
    name=results.index[0],
    marker=dict(color=Viridis[16])
)
mydata2 = go.Bar(
    x=results.loc['Accuracy'].index,
    y=results.loc['Accuracy'],
    name=results.index[1],
    marker=dict(color=Viridis[10])
)
mydata3 = go.Bar(
    x=results.loc['AUC score'].index,
    y=results.loc['AUC score'],
    name=results.index[2],
    marker=dict(color=Viridis[0])
)
mylayout = go.Layout(
    title='Comparison of Possible Models',
    xaxis = dict(title = 'Predictive models'), # x-axis label
    yaxis = dict(title = 'Score'), # y-axis label
    
)
fig = go.Figure(data=[mydata1, mydata2, mydata3], layout=mylayout)
iplot(fig)

Of the four models we tried, logistic regression has the highest ROC-AUC score, so we'll continue improving this model by tuning the hyperparameters and applying crossvalidation.

### Tuning the Logistic Regression model

In [20]:
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set(style="darkgrid", color_codes=None)

Gridsearch is a method to examine multiple versions of the same model, with different parameters, to determine which has the highest evaluation metrics. This cell is commented out, and its results pickled, because it is very time-consuming to run.

In [21]:
# # Create regularization penalty space (l1=ridge, l2=lasso)
# penalty = ['l1', 'l2'] 

# # Create regularization hyperparameter space
# C = np.logspace(0, 4, 10)

# # Create hyperparameter options
# hyperparameters = dict(C=C, penalty=penalty)

# # Create grid search using 5-fold cross validation
# grid_lr = GridSearchCV(LogisticRegression(), hyperparameters, cv=5,  n_jobs = 1, verbose=0)
# grid_lr.fit(X_train, y_train)
# print(grid_lr.best_params_)

In [22]:
# # Instantiate the model using those parameters
# log_model = grid_lr.best_estimator_

# # Pickle the results of gridsearch so we don't have to do that again!
# with open('logistic_best_params.pkl', 'wb') as f:
#     pickle.dump(log_model, f)
# f.close()

Our logistic regression model includes two important features to help avoid overfit:
* [L2 penalty](https://towardsdatascience.com/ridge-regression-for-better-usage-2f19b3a202db) (aka `Ridge Regression`). It penalizes larger coefficients, which are frequently associated with overfit.
* [C parameter](https://charleshsliao.wordpress.com/2017/05/20/logistic-regression-in-python-to-tune-parameter-c/) for regularization. C is actually the inverse of regularization strength (lambda) which shrinks larger coefficients towards zero. 


In [23]:
# Unpickle the results of gridsearch
pickle_off = open('logistic_best_params.pkl','rb')
log_model = pickle.load(pickle_off)
print(log_model)

LogisticRegression(C=1291.5496650148827, class_weight=None, dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)


In [24]:
# Predict on the testing data
predictions=log_model.predict(X_test)
probabilities = log_model.predict_proba(X_test)[:,1]

`scikit learn` provides several metrics for evaluating the precision and accuracy of our model. Let's combine all of them into a single reusable function.

In [25]:
# Full list of metrics
def model_metrics(y_test, predictions):
    '''
    Calculate 5 standard model metrics
    Return a dictionary with the metrics
    '''
    f1 = metrics.f1_score(y_test, predictions)
    accuracy = metrics.accuracy_score(y_test, predictions)
    error = 1 - accuracy
    precision = metrics.precision_score(y_test, predictions)
    recall = metrics.recall_score(y_test, predictions)
    rocauc =  metrics.roc_auc_score(y_test, predictions)
    return {'f1 score':f1, 'accuracy score': accuracy, 'error rate': error, 'precision score': precision, 'recall score': recall, 'ROC-AUC score': rocauc}

model_metrics(y_test, predictions)

{'ROC-AUC score': 0.7851862137719862,
 'accuracy score': 0.8901853367009356,
 'error rate': 0.10981466329906442,
 'f1 score': 0.010893944248638257,
 'precision score': 0.005490956072351421,
 'recall score': 0.68}

### ROC-AUC (Receiver Operating Characteristic - Area Under the Curve)

Let's take a deeper dive into the ROC-AUC score, and display it as a graphic. The larger the area below the curve, the better our model will be at predicting the outcomes, and avoid both false negatives and false positives.

In [26]:
FPR, TPR, _ = roc_curve(y_test, predictions)
roc_score=round(100*roc_auc_score(y_test, predictions),1)
roc_score

78.5

In [27]:
# ROC-AUC figure

data = [
    {
      'x':FPR, 
      'y':TPR, 
      'type':'scatter',
      'mode': 'lines',
      'name': 'AUC: '+str(roc_score)
      },
     {'x':[0,1], 
      'y':[0,1], 
      'type':'scatter',
      'mode': 'lines',
      'name': 'Baseline Area: 50.0'}]

layout = go.Layout(
    title = 'Receiver Operating Characteristic - Area Under Curve',
    width=650,
    height=600, 
)
fig = go.Figure(data=data, layout=layout)
iplot(fig)

Another way of evaluating our model is the `confusion matrix`, which displays actual counts for true positives, false negatives, etc.

In [28]:
# A confusion matrix tells us our false positives and false negatives:
matrix=confusion_matrix(y_test, predictions)
cm=pd.DataFrame(matrix, columns=['predicted: +', 'predicted: -'], index=['ground truth: +', 'ground truth: -'])
cm=cm.reset_index(drop=False)
cm=cm.rename(columns={'index': f'n = {len(y_test)}'})

In [29]:
import plotly.figure_factory as ff
table = ff.create_table(cm)
iplot(table)

In [44]:
# Pickle the results 
with open('confu_matrix6.pkl', 'wb') as f:
    pickle.dump(cm, f)
f.close()

## Summary of Evaluation Metrics

Accuracy:
Overall, how often is it correct?
(TP + TN) / total

Sensitivity (Recall):
When actual value is positive, how often is prediction correct?
TP / actual yes
“True Positive Rate” or “Recall”

Precision:
TP / (TP + FP)

Specificity:
When actual value is negative, how often is prediction correct?
TN / actual no

Misclassification Rate (Error Rate):
Overall, how often is it wrong?
(FP + FN) / total

## Feature importance

One of the advantages of logistic regression is that it can tell us not only which features are associated with the outcome, but the size and directionality of that association. This is called `feature importance`.

[Interpretation of the coefficients](http://kevinyuan.ca/2016/08/01/Interpreting-Logistic-Regression/) can be challenging

In [30]:
# Feature importance
results=pd.DataFrame(list(zip(X_test.columns, logreg.coef_[0])), columns=['feature', 'coefficient'])
results=results.sort_values(by='coefficient', ascending=False)

In [31]:
big_coeffs=results.head(15)

The intercept term from the logistic regression is the log odds of our base reference term.

In [51]:
# To interpret the coefficients, add them to the intercept and then read as odds ratios.
logreg.intercept_

array([-0.0998009])

Adding any other coefficient to the intercept gives the log odds of failure for that feature. For example, an increase of one unit in attribute 4 decreases the odds of failure (i.e., makes it more likely not to fail).

In [50]:
# List the regression coefficients from greatest to smallest.
big_coeffs

,feature,coefficient
3,attribute4,0.535093
21,attribute4_lag01,0.469254
13,attribute2_lag01,0.417921
1,attribute2,0.417046
22,attribute4_lag02,0.406024
14,attribute2_lag02,0.367693
23,attribute4_lag03,0.364203
33,attribute7_lag01,0.353935
6,attribute7,0.352449
15,attribute2_lag03,0.327986


In [33]:
# Let's display that with Plotly.
mydata = [go.Bar(
    x=big_coeffs['feature'],
    y=big_coeffs['coefficient'],
    marker=dict(color=Viridis[::-1])
)]

mylayout = go.Layout(
    title='Logistic Regression Results',
    xaxis = dict(title = 'Coefficients'), 
    yaxis = dict(title = 'Odds Ratio to Failure'), 

)
fig = go.Figure(data=mydata, layout=mylayout)
pio.write_image(fig, '../images/logistic.png')
iplot(fig)

#### Were these features highly correlated with the outcome?

In [34]:
cols=list(big_coeffs.feature)[:5]
cols.append('failure')
cols

['attribute4',
 'attribute4_lag01',
 'attribute2_lag01',
 'attribute2',
 'attribute4_lag02',
 'failure']

None of the features, by itself, has a high correlation with failure. This explains in part why our model doesn't have greater predictive power.

In [35]:
df[cols].corr()
# No, they were not.

,attribute4,attribute4_lag01,attribute2_lag01,attribute2,attribute4_lag02,failure
attribute4,1.000000,0.991320,0.209833,0.213120,0.983055,0.064157
attribute4_lag01,0.991320,1.000000,0.210570,0.210180,0.991663,0.056897
attribute2_lag01,0.209833,0.210570,1.000000,0.991008,0.207976,0.058904
attribute2,0.213120,0.210180,0.991008,1.000000,0.206883,0.059865
attribute4_lag02,0.983055,0.991663,0.207976,0.206883,1.000000,0.052150
failure,0.064157,0.056897,0.058904,0.059865,0.052150,1.000000


## How often are we accurate when aggregated to the device level?

The model predicts probability of failure on a daily basis, but we might also want to know about probability that a device will fail at any point in its life. A confusion matrix will help us determine whether our model successfully predicts failure over the life of a device.

In [36]:
y1_test=y1_test.reset_index(drop=True) # for the concat to work correctly, must have a clean index.
preds_df=pd.DataFrame(predictions, columns=['preds'])
combined_testdf=pd.concat([y1_test, preds_df], axis=1)

print('y_test: ', len(y1_test))
print('probabilities: ', len(preds_df))
print('combined: ', len(combined_testdf))

y_test:  28111
probabilities:  28111
combined:  28111


In [37]:
combined_testdf.head()

,device,failure,preds
0,S1F0S68M,0,0
1,Z1F1R76A,0,0
2,W1F16RA7,0,0
3,S1F131F6,0,0
4,W1F14GTK,0,0


In [38]:
aggdf=combined_testdf.groupby('device')[['device', 'failure', 'preds']].mean().reset_index(drop=False)
print(aggdf.shape)
aggdf.head()

(676, 3)


,device,failure,preds
0,S1F01E6Y,0.000000,0.000000
1,S1F01XDJ,0.000000,0.000000
2,S1F023H2,0.166667,0.833333
3,S1F02L38,0.000000,0.000000
4,S1F03YZM,0.012821,0.461538


In [39]:
aggdf['failed']=0
aggdf.loc[aggdf['failure']>0, 'failed']=1
aggdf['failed'].value_counts()

0    651
1     25
Name: failed, dtype: int64

In [40]:
aggdf['pred_failed']=0
aggdf.loc[aggdf['preds']>0, 'pred_failed']=1
aggdf['pred_failed'].value_counts()

0    536
1    140
Name: pred_failed, dtype: int64

In [41]:
# Confusion Matrix (device level)
pd.crosstab(aggdf['failed'], aggdf['pred_failed'],  margins=True)

pred_failed,0,1,All
failed,,,
0,528,123,651
1,8,17,25
All,536,140,676


The classification model does a poor job of identifying true negatives. It only identifies 13% of failures.

## Save the datafile

In [43]:
combined_testdf.to_csv('../data/dataset6.gz', compression='gzip', index=False)
print(df.shape)

(93701, 39)
